In [16]:
import pandas as pd


In [17]:
df=pd.read_csv("/home/abrar/Desktop/Abrar/LangChain/08_Tools/Assignment/holdings.csv")

In [18]:
df.head()

,AsOfDate,OpenDate,CloseDate,ShortName,PortfolioName,StrategyRefShortName,Strategy1RefShortName,Strategy2RefShortName,CustodianName,DirectionName,...,StartPrice,Price,StartFXRate,FXRate,MV_Local,MV_Base,PL_DTD,PL_QTD,PL_MTD,PL_YTD
0,01/08/23,04/03/20,NaN,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,...,96.0,96.0,1.33,1.33,568320.00,7.558656e+05,92.5040,10833.7294,92.5040,41054.5854
1,01/08/23,04/03/20,NaN,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,...,96.0,96.0,1.33,1.33,84.48,1.123584e+02,0.0138,1.6104,0.0138,6.1027
2,01/08/23,04/03/20,NaN,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,...,96.0,96.0,1.33,1.33,756000.00,1.005480e+06,123.0523,14411.4221,123.0523,54612.3074
3,01/08/23,04/03/20,NaN,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,...,96.0,96.0,1.33,1.33,484800.00,6.447840e+05,78.9097,9241.6104,78.9097,35021.2257
4,01/08/23,04/03/20,NaN,Heather,Heather,Default,Asset,DefaultS2,Well Prime,Long,...,96.0,96.0,1.33,1.33,487680.00,6.486144e+05,79.3785,9296.5110,79.3785,35229.2726


In [19]:
df2=pd.read_csv("/home/abrar/Desktop/Abrar/LangChain/08_Tools/Assignment/trades.csv")

In [20]:
df2.head()

,id,RevisionId,AllocationId,TradeTypeName,SecurityId,SecurityType,Name,Ticker,CUSIP,ISIN,...,AllocationFees,AllocationCash,PortfolioName,CustodianName,StrategyName,Strategy1Name,Strategy2Name,Counterparty,AllocationRule,IsCustomAllocation
0,3489863,2,3460886,Buy,270471,Equity,Berry Brand 4/11 Equity,NaN,NaN,NaN,...,2800.00,7.002800e+06,HoldCo 1,JP MORGAN SECURITIES LLC,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 1,1
1,3489864,1,3460887,Sell,270471,Equity,Berry Brand 4/11 Equity,NaN,NaN,NaN,...,128.80,6.999871e+06,HoldCo 1,JP MORGAN SECURITIES LLC,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 1,0
2,3496826,1,3462756,Sell,290063,Equity,META-US,META,30303M102,US30303M1027,...,46985.99,2.553540e+09,HoldCo 3,CITIGROUP GLOBAL MARKETS INC.,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 3,0
3,3496828,3,3462769,Buy,290067,Equity,SPOT-US,SPOT,NaN,LU1778762911,...,20.20,1.098249e+06,HoldCo 11,Goldman Sachs International,Default,Asset,DefaultS2,ABGS,Single Fund Rule - HoldCo 11,1
4,3496829,4,3462770,Buy,290067,Equity,SPOT-US,SPOT,NaN,LU1778762911,...,60.62,3.294749e+06,HoldCo 11,Goldman Sachs International,Default,Asset,DefaultS2,ABGS,Single Fund Rule - HoldCo 11,1


In [21]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool,InjectedToolArg
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage,ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Annotated
import json
import smtplib
from email.mime.text import MIMEText
from pydantic import BaseModel, Field
import os

load_dotenv()

model=ChatOpenAI(model='gpt-4o-mini')

In [29]:
from typing import Literal

@tool("count_number_of_holdings_or_trades",description="count number of holdings or trades for a given fund")
def countHoldings(type:Literal["holding","trades"],portfolio:str)->int:
    type=type.lower().strip()
    portfolio=portfolio.strip()
    if type=="holding":
        return int(len(df[df["PortfolioName"]==portfolio]))
    elif type=="trades":
        return int(len(df2[df2["PortfolioName"]==portfolio]))
    else:
        return 0
    
    

In [57]:
from langchain.tools import tool

@tool(
    "get_fund_performance_by_yearly_pl",
    description=(
        "Calculate fund performance based on yearly Profit and Loss (PL_YTD). "
        "Returns funds ranked by total yearly P&L in descending order."
    )
)
def getFundPerformance() -> dict:
    filtered_df = df.dropna(subset=["PortfolioName", "PL_YTD"])

    if filtered_df.empty:
        return {"fund_performance": []}
    performance_df = (
        filtered_df
        .groupby("PortfolioName")["PL_YTD"]
        .sum()
        .reset_index()
        .sort_values(by="PL_YTD", ascending=False)
    )

    result = [
        {
            "fund": row["PortfolioName"],
            "pl_ytd": float(row["PL_YTD"])
        }
        for _, row in performance_df.iterrows()
    ]

    return {"fund_performance": result}


In [58]:
llm_with_tools=model.bind_tools([countHoldings,getFundPerformance])


In [154]:
user_query=input("How can i Assist you today")

messages=[HumanMessage(user_query)]


How can i Assist you today How many holding does the Abrar fund have


In [155]:
test_result=llm_with_tools.invoke(messages)

In [145]:
print(test_result)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 116, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-D00rhxVjv3JCh8XSbAEdrCcXkGWor', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019bda66-4f6c-72b2-aa45-f3f8d747f95b-0' tool_calls=[{'name': 'get_fund_performance_by_yearly_pl', 'args': {}, 'id': 'call_Mb9V1e1f7SzKMRiYAp8L3MYG', 'type': 'tool_call'}] usage_metadata={'input_tokens': 116, 'output_tokens': 17, 'total_tokens': 133, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [156]:

import json

messages.append(test_result)


for tool_call in test_result.tool_calls:

    # Part 1: Count holdings / trades
    if tool_call["name"] == "count_number_of_holdings_or_trades":

        count = countHoldings.invoke(tool_call["args"])

        messages.append(
            ToolMessage(
                content=str(count),  # MUST be string
                tool_call_id=tool_call["id"]
            )
        )

    # Part 2: Fund performance by yearly P&L
    elif tool_call["name"] == "get_fund_performance_by_yearly_pl":

        performance = getFundPerformance.invoke({})

        messages.append(
            ToolMessage(
                content=json.dumps(performance),  # structured JSON as string
                tool_call_id=tool_call["id"]
            )
        )


In [157]:
result=llm_with_tools.invoke(messages)
messages.append(result)

In [158]:
print(result.content)

The Abrar fund has 0 holdings.
